# CARGA DOS DADOS DE DESPESA DO PORTAL DADOS MG

Script notebook para carga inicial de dados do portal [dados.mg](https://dados.mg.gov.br/dataset/despesa) para criação do spreadmart financeiro da Assessoria de Dados da SCPO/SEPLAG
  
  Desenvolvedor: Andrey Morais Labanca  
  Contato: moraislabanca@gmail.com  

## TO DO
1. Clausulas SQL para somente criar ou deletar tabelas caso não já exisitam
1. Tratamento de erros e exceções para quando não for possível criar ou conectar na base de dados, criar ou deletar tabelas e etc.
1. Atualmente usando pandas para concatenação de tabelas. Ineficiente e utiliza muita memória. Necessário trocar para algo relativo a SQL, DuckDB ou que vá concantenando parcialmente em disco sem carregar tudo para memória.

## Imports, constants e setup

In [1]:
import pandas as pd
import glob
import time
import duckdb
import os

DB_NAME = 'database/dadosmg.db'
CSV_PATH = 'datasets/'

# obtem lista de paths para arquivos CSV localizados no caminho CSV_PATH
file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}*.csv'))]

# paths de bases csv que sao separadas por anos
file_paths_desp = [x for x in file_paths if "dm_empenho_desp_" in x] 
file_paths_ft = [x for x in file_paths if "ft_despesa_" in x] 

# paths de bases csv que não são separadas por anos
file_paths = list(set(file_paths) - set(file_paths_desp) - set(file_paths_ft))


con = duckdb.connect(DB_NAME) #Cria se não existe e se conecta à base de dados

## Importa dados de despesa e cria tabelas
### Exclui bases que são separadas por anos.

In [8]:


#Importa CSV das bases não separadas por anos e cria respectivas tabelas
for file in file_paths:
    _ , tail = os.path.split(file)
    name , file_extension = os.path.splitext(tail)
    con.execute(f"""CREATE TABLE '{name}' AS SELECT * FROM read_csv_auto('{file}')""")
    print(f"Arquivo {file} carregado para tabela {name}")

['datasets/ft_despesa_2002.csv',
 'datasets/ft_despesa_2003.csv',
 'datasets/ft_despesa_2004.csv',
 'datasets/ft_despesa_2005.csv',
 'datasets/ft_despesa_2006.csv',
 'datasets/ft_despesa_2007.csv',
 'datasets/ft_despesa_2008.csv',
 'datasets/ft_despesa_2009.csv',
 'datasets/ft_despesa_2010.csv',
 'datasets/ft_despesa_2011.csv',
 'datasets/ft_despesa_2012.csv',
 'datasets/ft_despesa_2013.csv',
 'datasets/ft_despesa_2014.csv',
 'datasets/ft_despesa_2015.csv',
 'datasets/ft_despesa_2016.csv',
 'datasets/ft_despesa_2018.csv',
 'datasets/ft_despesa_2020.csv',
 'datasets/ft_despesa_2022.csv',
 'datasets/ft_despesa_2023.csv']

## Descreve tabelas criadas

In [5]:
con.sql("""DESCRIBE""")

┌─────────────────────┬───────────────────────────────────┬────────────────────────────────────────────────┬───────────┐
│     table_name      │           column_names            │                  column_types                  │ temporary │
│       varchar       │             varchar[]             │                   varchar[]                    │  boolean  │
├─────────────────────┼───────────────────────────────────┼────────────────────────────────────────────────┼───────────┤
│ dm_acao             │ [id_acao, ano_exercicio, cd_aca…  │ [BIGINT, BIGINT, BIGINT, VARCHAR]              │ false     │
│ dm_elemento_desp    │ [id_elemento, cd_elemento, nome]  │ [BIGINT, BIGINT, VARCHAR]                      │ false     │
│ dm_fonte            │ [id_fonte, cd_fonte, nome]        │ [BIGINT, BIGINT, VARCHAR]                      │ false     │
│ dm_funcao_desp      │ [id_funcao, ano_exercicio, cd_f…  │ [BIGINT, BIGINT, BIGINT, VARCHAR]              │ false     │
│ dm_item_desp        │ [id_item

## Agrega arquivos de empenho anuais e cria tabela

In [2]:
tbl_agg_name = 'dm_empenho_desp'
df_agg = pd.DataFrame()
num_linhas = 0

for file in file_paths_desp:
    print(f'Lendo:', file)
    df1 = con.execute(f"""SELECT * FROM '{file}' """).df()
    df_agg = pd.concat([df_agg, df1])
    num_linhas += len(df1)
      
con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """) 
print(f'Tabela {tbl_agg_name} criada')

print('Total de linhas tabelas lidas:', num_linhas)
print('Total de linhas do dataframe concatenado:', len(df_agg))
print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

con.table(tbl_agg_name).show()

NameError: name 'file_paths_desp' is not defined

## Agrega arquivos de faturamento anuais e cria tabela

In [ ]:
tbl_agg_name = 'ft_despesa'
df_agg = pd.DataFrame()
num_linhas = 0

for file in file_paths_ft:
    print(f'Lendo:', file)
    df1 = con.execute(f"""SELECT * FROM '{file}' """).df()
    df_agg = pd.concat([df_agg, df1])
    num_linhas += len(df1)
      
con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """) 
print(f'Tabela {tbl_agg_name} criada')

print('Total de linhas tabelas lidas:', num_linhas)
print('Total de linhas do dataframe concatenado:', len(df_agg))
print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

con.table(tbl_agg_name).show()

Lendo: datasets/ft_despesa_2002.csv
Lendo: datasets/ft_despesa_2003.csv
Lendo: datasets/ft_despesa_2004.csv
Lendo: datasets/ft_despesa_2005.csv
Lendo: datasets/ft_despesa_2006.csv
Lendo: datasets/ft_despesa_2007.csv
Lendo: datasets/ft_despesa_2008.csv
Lendo: datasets/ft_despesa_2009.csv
Lendo: datasets/ft_despesa_2010.csv
Lendo: datasets/ft_despesa_2011.csv
Lendo: datasets/ft_despesa_2012.csv
Lendo: datasets/ft_despesa_2013.csv
Lendo: datasets/ft_despesa_2014.csv
Lendo: datasets/ft_despesa_2015.csv
Lendo: datasets/ft_despesa_2016.csv
Lendo: datasets/ft_despesa_2018.csv
Lendo: datasets/ft_despesa_2020.csv
Lendo: datasets/ft_despesa_2022.csv
Lendo: datasets/ft_despesa_2023.csv
